In [187]:
# Imorting libraries 

import requests
import pandas as pd 
import json
import calendar
import hvplot.pandas
from pathlib import Path
import seaborn as sns
%matplotlib inline


series_list1 = {
'LAUST010000000000003':'Alabama', 'LAUST020000000000003':'Alaska', 'LAUST040000000000003':'Arizona', 'LAUST050000000000003':'Arkansas', 'LAUST060000000000003':'California', 
'LAUST080000000000003':'Colorado', 'LAUST090000000000003':'Connecticut', 'LAUST100000000000003':'Delaware', 'LAUST120000000000003':'Florida', 'LAUST130000000000003':'Georgia', 
'LAUST150000000000003':'Hawaii', 'LAUST160000000000003':'Idaho', 'LAUST170000000000003':'Illinois', 'LAUST180000000000003':'Indiana', 
'LAUST190000000000003':'Iowa', 'LAUST200000000000003':'Kansas', 'LAUST210000000000003':'Kentucky', 'LAUST220000000000003':'Louisiana', 'LAUST230000000000003':'Maine', 
'LAUST240000000000003':'Maryland', 'LAUST250000000000003':'Massachusetts', 'LAUST260000000000003':'Michigan', 'LAUST270000000000003':'Minnesota', 'LAUST280000000000003':'Mississippi',
'LAUST290000000000003':'Missouri'} 
series_list2 = {
'LAUST300000000000003':'Montana', 'LAUST310000000000003':'Nebraska', 'LAUST320000000000003':'Nevada', 'LAUST330000000000003':'New Hampshire', 'LAUST340000000000003':'New Jersey', 
'LAUST350000000000003':'New Mexico', 'LAUST360000000000003':'New York', 'LAUST370000000000003':'North Carolina', 'LAUST380000000000003':'North Dakota', 'LAUST390000000000003':'Ohio', 'LAUST400000000000003':'Oklahoma', 
'LAUST410000000000003':'Oregon', 'LAUST420000000000003':'Pennsylvania', 'LAUST440000000000003':'Rhode Island', 'LAUST450000000000003':'South Carolina', 'LAUST460000000000003':'South Dakota', 'LAUST470000000000003':'Tennessee', 
'LAUST480000000000003':'Texas', 'LAUST490000000000003':'Utah', 'LAUST500000000000003':'Vermont', 'LAUST510000000000003':'Virginia', 
'LAUST530000000000003':'Washington', 'LAUST540000000000003':'West Virginia', 'LAUST550000000000003':'Wisconsin', 'LAUST560000000000003':'Wyoming',
}
df = pd.DataFrame(columns= ["State","Year","Month","Unemployment Rate"])


In [188]:
headers = {'Content-type': 'application/json'}
data1 = json.dumps({"seriesid": list(series_list1.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data1, headers=headers)
json_data1 = json.loads(p.text)
data2 = json.dumps({"seriesid": list(series_list2.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data2, headers=headers)
json_data2 = json.loads(p.text)

for series in json_data1['Results']['series']:
    state = series_list1[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)

for series in json_data2['Results']['series']:
    state = series_list2[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)
df

,State,Year,Month,Unemployment Rate
0,Alabama,2021,December,2.6
1,Alabama,2021,November,2.6
2,Alabama,2021,October,2.9
3,Alabama,2021,September,3.0
4,Alabama,2021,August,3.7
...,...,...,...,...
1795,Wyoming,2019,May,3.3
1796,Wyoming,2019,April,3.4
1797,Wyoming,2019,March,4.0
1798,Wyoming,2019,February,3.9


In [189]:
df_2021 = df.loc[df['Year'] == '2021'].copy()
df_2021['Year Month'] = df_2021['Year'] +' '+ df_2021['Month']
df_2021 = df_2021[['Year Month', 'State','Unemployment Rate']]
df_2021.set_index('Year Month')

,State,Unemployment Rate
Year Month,,
2021 December,Alabama,2.6
2021 November,Alabama,2.6
2021 October,Alabama,2.9
2021 September,Alabama,3.0
2021 August,Alabama,3.7
...,...,...
2021 May,Wyoming,4.7
2021 April,Wyoming,5.0
2021 March,Wyoming,5.4


In [190]:
df_2021.sort_index(inplace=True,ascending=False)
df_2021 = df_2021.astype({'Unemployment Rate':'float'})
df_2021.hvplot(y='Unemployment Rate', x = 'Year Month',groupby='State', rot = 90)

:DynamicMap   [State]
   :Curve   [Year Month]   (Unemployment Rate)

In [191]:
# Setting up the path to the CSV file
csv_path = Path("../FInCen CSV Data/FinCen SAR Stats 2021 (Months).csv")

# Reading the csv file
csv_file = pd.read_csv(csv_path, index_col = "Year Month", thousands=',')
csv_file.head()


,State,Industry,Suspicious Activity,Count
Year Month,,,,
2021 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
2021 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Other Fraud (Type),2
2021 January,Alabama,Casino/Card Club - Tribal Authorized Casino,[Total],4
2021 January,Alabama,Depository Institution,ACH,286
2021 January,Alabama,Depository Institution,Advance Fee,3


In [192]:
#Using "loc" to remove the lines that have "Totals"

csv_file = csv_file.loc[csv_file["Industry"]!="[Total]" ]
csv_file = csv_file.loc[csv_file["Suspicious Activity"]!="[Total]" ]
csv_file = csv_file.loc[csv_file["State"]!="[Total]" ]
csv_file.head()

,State,Industry,Suspicious Activity,Count
Year Month,,,,
2021 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
2021 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Other Fraud (Type),2
2021 January,Alabama,Depository Institution,ACH,286
2021 January,Alabama,Depository Institution,Advance Fee,3
2021 January,Alabama,Depository Institution,Business Loan,7


In [193]:
csv_file = csv_file.reset_index()
grouped_stats = csv_file[["Year Month", "State", "Count"]].copy()

grouped_stats = grouped_stats.groupby(["Year Month", "State"]).sum()
grouped_stats

Count
Year Month     State                 
2021 April     APO / DPO / FPO      7
               Alabama           1918
               Alaska              97
               American Samoa       1
               Arizona            979
...                               ...
2021 September Virginia         10999
               Washington        1901
               West Virginia      686
               Wisconsin          997
               Wyoming             87

[697 rows x 1 columns]

In [194]:

df_2021 = df_2021.reset_index()
grouped_stats = grouped_stats.reset_index()
df_2021 = df_2021.sort_values("State")

combined_df = pd.concat([df_2021, grouped_stats], axis="columns", join="inner")
combined_df


,index,Year Month,State,Unemployment Rate,Year Month,State,Count
599,0,2021 December,Alabama,2.6,2021 October,Indiana,1336
588,11,2021 January,Alabama,4.1,2021 October,California,23835
589,10,2021 February,Alabama,4.0,2021 October,Colorado,4871
590,9,2021 March,Alabama,3.6,2021 October,Connecticut,687
591,8,2021 April,Alabama,3.3,2021 October,Delaware,8558
...,...,...,...,...,...,...,...
9,1766,2021 October,Wyoming,3.5,2021 April,Delaware,6930
10,1765,2021 November,Wyoming,3.5,2021 April,District of Columbia,197
11,1764,2021 December,Wyoming,3.2,2021 April,Florida,6571
6,1769,2021 July,Wyoming,4.3,2021 April,California,19613
